# import

In [1]:
import pandas as pd
import openpyxl
import tensorflow as tf
import numpy as np
import os, sys




2024-11-14 00:05:41.033834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 00:05:41.033858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 00:05:41.034575: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 00:05:41.038823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 00:05:41.653733: W tensorflow/compiler/tf2

In [2]:
sys.path.append('/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/')
print(sys.path)
from models import get_data, utils, models_pre, models_train, reports_build

['/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels', '/home/jczars/anaconda3/envs/tf/lib/python310.zip', '/home/jczars/anaconda3/envs/tf/lib/python3.10', '/home/jczars/anaconda3/envs/tf/lib/python3.10/lib-dynload', '', '/home/jczars/.local/lib/python3.10/site-packages', '/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages', '/media/jczars/4C22F02A22F01B22/$WinREAgent/Pollen_classification_view/']


/home/jczars/anaconda3/envs/tf/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
workbook_path = '/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/config_pseudo_label_pre.xlsx'
id_test = 5

# config_data

In [12]:
# Load configuration data from 'Sheet'
config_data = pd.read_excel(workbook_path, sheet_name="Sheet")

In [13]:
config = config_data.loc[id_test]
config

id_test                      5
base                      BI_5
path_base              BD/BI_5
aug                        sem
img_size                   224
batch_size                   4
epochs                     200
alpha                  0.00001
model              DenseNet201
last_activation        softmax
activation                relu
dense_size                1024
dropout                    0.3
freeze                       0
split_valid                0.2
optimizer              RMSprop
learning_rate           0.0001
type_model            imagenet
limiar                   0.995
Name: 5, dtype: object

# functions

## train_model

In [14]:
def train_model(config, train_data, val_data, time_step):
    """
    Train a model with the given configuration and data. If time_step > 0, it will load a pre-trained model
    and continue training. Otherwise, it will train a new model from scratch.

    Parameters
    ----------
    config : dict
        A dictionary containing the configuration for model training.
    train_data : tuple
        A tuple containing the training data and labels.
    val_data : tuple
        A tuple containing the validation data and labels.
    time_step : int
        The current training step. If time_step > 0, it will load a model from the previous step.

    Returns
    -------
    model_inst : keras.Model
        The trained or reloaded model instance.
    res_train : dict
        A dictionary containing the training history and metrics.
    """
    print('\n[INFO]--> time_step ', time_step)
    
    # Reset model_inst to ensure it starts fresh for each time step
    model_inst = None
    
    # If time_step > 0, try to load the model from a previous step, otherwise train a new model
    if time_step > 0:
        # Build the model path for the previous step
        model_name = f"{config['id_test']}_{config['model']}_bestLoss_{time_step - 1}.keras"
        save_path = os.path.join(config['save_dir'], model_name)
        
        # Load the model from the previous time step
        if os.path.exists(save_path):
            print(f"[INFO]--> Loading model from {save_path}")
            model_inst = tf.keras.models.load_model(save_path)
            
            # Explicitly freeze the layers again
            for layer in model_inst.layers:
                layer.trainable = False  # Freeze all layers
            
            # Optionally unfreeze the last few layers if needed (based on config['freeze'])
            for i, layer in enumerate(model_inst.layers):
                if i >= config['freeze']:
                    layer.trainable = True  # Unfreeze layers after the specified freeze index
            
            print(f"[INFO]--> Model layers frozen up to layer {config['freeze']}")
        else:
            raise ValueError(f"[ERROR]--> Model from time_step {time_step - 1} not found at {save_path}")
    
    # else:
    #     # Instantiate the model from scratch for time_step == 0
    #     print("[INFO]--> Training a new model from scratch...")
    #     model_inst = models_pre.hyper_model_up(config, verbose=1)
    
    # Train the model with the training and validation data
    res_train = models_train.run_train(train_data, val_data, model_inst, config)
    
    # Save the model at the current time step
    model_name = f"{config['id_test']}_{config['model']}_bestLoss_{time_step}.keras"
    save_path = os.path.join(config['save_dir'], model_name)
    
    # Save the trained model
    model_inst.save(save_path)
    print(f"[INFO]--> Model saved at {save_path}")
    
    return model_inst, res_train

## recupera csv

In [15]:
rec_csv = pd.read_excel(workbook_path, sheet_name="Table")
rec_csv

,Tempo,test_loss,test_accuracy,precision,recall,fscore,kappa,str_time,end_time,delay,best_epoch,ini,select,rest,train,new_train,id_test
0,0,0.175673,0.952756,0.956,0.953,0.953,0.936,2024-11-13 18:42:26,2024-11-13 18:50:03,00:07:37,39,3273,2308,965,406,2714,5
1,1,0.154442,0.960630,0.962,0.961,0.960,0.947,2024-11-13 18:52:39,2024-11-13 18:57:18,00:04:39,16,965,233,732,2714,2947,5
2,2,0.275366,0.937008,0.940,0.937,0.937,0.915,2024-11-13 18:59:16,2024-11-13 19:06:37,00:07:21,1,732,247,485,2947,3194,5
3,3,0.322806,0.944882,0.947,0.945,0.944,0.926,2024-11-13 19:08:35,2024-11-13 19:19:10,00:10:35,6,485,157,328,3194,3351,5
4,4,0.354300,0.929134,0.938,0.929,0.929,0.904,2024-11-13 19:21:07,2024-11-13 19:30:06,00:08:59,2,328,96,232,3351,3447,5
5,5,0.296923,0.944882,0.947,0.945,0.945,0.926,2024-11-13 19:32:03,2024-11-13 19:43:35,00:11:32,7,232,32,200,3447,3479,5
6,6,0.367513,0.952756,0.956,0.953,0.953,0.936,2024-11-13 19:45:33,2024-11-13 19:57:22,00:11:49,8,200,49,151,3479,3528,5
7,7,0.333440,0.952756,0.956,0.953,0.953,0.936,2024-11-13 19:59:19,2024-11-13 20:10:33,00:11:14,6,151,20,131,3528,3548,5
8,8,0.397848,0.952756,0.956,0.953,0.953,0.936,2024-11-13 20:12:31,2024-11-13 20:21:18,00:08:47,1,131,25,106,3548,3573,5
9,9,0.474831,0.952756,0.956,0.953,0.953,0.936,2024-11-13 20:23:17,2024-11-13 20:36:46,00:13:29,10,106,11,95,3573,3584,5


In [19]:
fil=rec_csv[rec_csv['id_test'] == id_test]
fil

,Tempo,test_loss,test_accuracy,precision,recall,fscore,kappa,str_time,end_time,delay,best_epoch,ini,select,rest,train,new_train,id_test
0,0,0.175673,0.952756,0.956,0.953,0.953,0.936,2024-11-13 18:42:26,2024-11-13 18:50:03,00:07:37,39,3273,2308,965,406,2714,5
1,1,0.154442,0.960630,0.962,0.961,0.960,0.947,2024-11-13 18:52:39,2024-11-13 18:57:18,00:04:39,16,965,233,732,2714,2947,5
2,2,0.275366,0.937008,0.940,0.937,0.937,0.915,2024-11-13 18:59:16,2024-11-13 19:06:37,00:07:21,1,732,247,485,2947,3194,5
3,3,0.322806,0.944882,0.947,0.945,0.944,0.926,2024-11-13 19:08:35,2024-11-13 19:19:10,00:10:35,6,485,157,328,3194,3351,5
4,4,0.354300,0.929134,0.938,0.929,0.929,0.904,2024-11-13 19:21:07,2024-11-13 19:30:06,00:08:59,2,328,96,232,3351,3447,5
5,5,0.296923,0.944882,0.947,0.945,0.945,0.926,2024-11-13 19:32:03,2024-11-13 19:43:35,00:11:32,7,232,32,200,3447,3479,5
6,6,0.367513,0.952756,0.956,0.953,0.953,0.936,2024-11-13 19:45:33,2024-11-13 19:57:22,00:11:49,8,200,49,151,3479,3528,5
7,7,0.333440,0.952756,0.956,0.953,0.953,0.936,2024-11-13 19:59:19,2024-11-13 20:10:33,00:11:14,6,151,20,131,3528,3548,5
8,8,0.397848,0.952756,0.956,0.953,0.953,0.936,2024-11-13 20:12:31,2024-11-13 20:21:18,00:08:47,1,131,25,106,3548,3573,5
9,9,0.474831,0.952756,0.956,0.953,0.953,0.936,2024-11-13 20:23:17,2024-11-13 20:36:46,00:13:29,10,106,11,95,3573,3584,5


In [21]:
print(len(fil)-1)

13


In [22]:
tempo_px=len(fil)

In [24]:
# recurarar csv_NewtainSet14
_pseudo_csv="/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/"

_csv_New_TrainSet = os.path.join(_pseudo_csv, f'trainSet_T{tempo_px}.csv')
print(_csv_New_TrainSet)

/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/trainSet_T14.csv


In [26]:
os.path.isfile(_csv_New_TrainSet)

True

In [28]:
def arquivo_existe(caminho_arquivo):
    """
    Verifica se um arquivo existe no sistema de arquivos.
    
    Args:
        caminho_arquivo (str): Caminho completo do arquivo.
    
    Returns:
        str: Mensagem de erro ou confirmação.
    """
    if os.path.isfile(caminho_arquivo):
        return f"O arquivo '{caminho_arquivo}' existe!"
    else:
        return f"Erro: O arquivo '{caminho_arquivo}' não existe!"

## New Train

In [29]:
arquivo_existe(_csv_New_TrainSet)

"O arquivo '/media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/trainSet_T14.csv' existe!"

In [25]:
verificar = pd.read_csv(_csv_New_TrainSet)
verificar

,file,labels,confidence
0,BD/BI_5/labels/equatorial_alongada/3.Castanea ...,equatorial_alongada,NaN
1,BD/BI_5/labels/polar_tricircular/2.Erica (123)...,polar_tricircular,NaN
2,BD/BI_5/labels/equatorial_alongada/15.Ferula (...,equatorial_alongada,NaN
3,BD/BI_5/labels/polar_tricircular/2.Erica (104)...,polar_tricircular,NaN
4,BD/BI_5/labels/equatorial_alongada/3.Castanea ...,equatorial_alongada,NaN
...,...,...,...
3608,BD/BI_5/images_unlabels/unlabels/20.Olea (16).png,polar_triangular,0.999949
3609,BD/BI_5/images_unlabels/unlabels/20.Olea (514)...,equatorial_circular,0.997482
3610,BD/BI_5/images_unlabels/unlabels/20.Olea (691)...,polar_triangular,0.999784
3611,BD/BI_5/images_unlabels/unlabels/4.Eucalyptus ...,polar_triangular,0.998355


In [31]:
os.chdir("/media/jczars/4C22F02A22F01B22/Pollen_classification_view")

In [32]:
train, val = get_data.reload_data_train(config, _csv_New_TrainSet)

training_data_path  /media/jczars/4C22F02A22F01B22/Pollen_classification_view/0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5/pseudo_csv/trainSet_T14.csv

 ######## Data Generator ################
Found 2891 validated image filenames belonging to 6 classes.
Found 722 validated image filenames belonging to 6 classes.


## build

In [34]:
save_dir = f'0_pseudo_labels/Reports/{id_test}_{config["model"]}_{config["aug"]}_{config["base"]}'
save_dir

'0_pseudo_labels/Reports/5_DenseNet201_sem_BI_5'

In [36]:
config_train={
    'id_test': id_test,
    'model': config['model'],
    'save_dir': save_dir,
    'freeze': config['freeze'],
    'batch_size': config['batch_size'],
    'epochs': config['epochs'],
    }

KeyError: 'verbose'

In [33]:
model_train, res_train = train_model(config_train, train, val, tempo_px)


[INFO]--> time_step  14


KeyError: 'save_dir'